In [32]:
import tabula
import pandas as pd
import numpy as np

In [60]:
!pip install tabula

You should consider upgrading via the 'c:\users\shiela.m.sandoval\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


### Get the data and read PDF

In [33]:
file = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/935287/2020-11-13_Tier_2_5_Register_of_Sponsors.pdf'

### Parse the Table on the First Page

Only the lower 75% of the first page contains a table, so parsing is done a bit differently.

In [34]:
tables = tabula.read_pdf(file, pages = "1", multiple_tables = True, area=[200,10,800,830], pandas_options={'header': None})

In [35]:
columns = ['Organization Name', 'Town/City', 'County', 'Tier & Rating', 'Sub Tier']

In [36]:
first_page = tables[0]
first_page.columns = columns

In [37]:
total_pages_cell = first_page['Town/City'].iloc[len(first_page['Town/City']) - 1]
total_pages = int(total_pages_cell.split()[-1])
total_pages

2108

### Parse the other pages

In [38]:
# other_pages = tabula.read_pdf(file, pages = '2-' + str(total_pages), multiple_tables = True, pandas_options={'header': None})
other_pages = tabula.read_pdf(file, pages = '2-10', multiple_tables = True, area=[30,10,800,830], pandas_options={'header': None})

In [39]:
new_df = pd.concat(other_pages)
new_df.columns = columns

In [40]:
all_data = pd.concat([first_page, new_df], axis=0).reset_index()

In [41]:
all_data.head()

,index,Organization Name,Town/City,County,Tier & Rating,Sub Tier
0,0,@ Home Accommodation Services Ltd,London,NaN,NaN,NaN
1,1,NaN,NaN,NaN,Tier 2 (A rating),Tier 2 General
2,2,NaN,NaN,NaN,Tier 5 (A rating),Creative & Sporting
3,3,]performance s p a c e [,Folkestone,Kent,NaN,NaN
4,4,NaN,NaN,NaN,Tier 5 (A rating),Creative & Sporting


### Cleaning the data

- Remove the rows containing the Page number
- Fill blank cells with the data from previous row (fillna: method='ffill')
- Drop duplicate rows without value in Tier & Rating column

In [42]:
all_data['Town/City'] = all_data['Town/City'].replace(to_replace=r'Page', value=np.nan, regex=True)

In [43]:
all_data['Organization Name'] = all_data['Organization Name'].fillna(method='ffill')
all_data['Town/City'] = all_data['Town/City'].fillna(method='ffill')
# all_data['County'] = all_data['County'].fillna(method='ffill')

In [44]:
all_data[:20]

,index,Organization Name,Town/City,County,Tier & Rating,Sub Tier
0,0,@ Home Accommodation Services Ltd,London,NaN,NaN,NaN
1,1,@ Home Accommodation Services Ltd,London,NaN,Tier 2 (A rating),Tier 2 General
2,2,@ Home Accommodation Services Ltd,London,NaN,Tier 5 (A rating),Creative & Sporting
3,3,]performance s p a c e [,Folkestone,Kent,NaN,NaN
4,4,]performance s p a c e [,Folkestone,Kent,Tier 5 (A rating),Creative & Sporting
5,5,012 Global Ltd,London,Kent,NaN,NaN
6,6,012 Global Ltd,London,Kent,Tier 2 (A rating),Tier 2 General
7,7,0-two Maintenance,London,Kent,NaN,NaN
8,8,0-two Maintenance,London,Kent,Tier 2 (A rating),Tier 2 General
9,9,1 Digitals Europe Limited,Slough,Kent,NaN,NaN


In [45]:
all_data = all_data.drop(all_data.loc[all_data['Tier & Rating'].isna()].index, axis=0).drop(['index'], axis=1)
all_data = all_data.reset_index(drop=True)

In [46]:
all_data.head()

,Organization Name,Town/City,County,Tier & Rating,Sub Tier
0,@ Home Accommodation Services Ltd,London,NaN,Tier 2 (A rating),Tier 2 General
1,@ Home Accommodation Services Ltd,London,NaN,Tier 5 (A rating),Creative & Sporting
2,]performance s p a c e [,Folkestone,Kent,Tier 5 (A rating),Creative & Sporting
3,012 Global Ltd,London,Kent,Tier 2 (A rating),Tier 2 General
4,0-two Maintenance,London,Kent,Tier 2 (A rating),Tier 2 General


In [52]:
# Clean character casing
def clean_char_cases(column_data):
    lower_case = list(column_data.str.lower())
    
    for i, t in enumerate(lower_case):
        lower_case[i] = ' '.join(list(map(lambda x: x.capitalize(), t.split())))
        
    return lower_case

In [55]:
all_data['Town/City'] = clean_char_cases(all_data['Town/City'])

In [56]:
all_data.head()

,Organization Name,Town/City,County,Tier & Rating,Sub Tier
0,@ Home Accommodation Services Ltd,London,NaN,Tier 2 (A rating),Tier 2 General
1,@ Home Accommodation Services Ltd,London,NaN,Tier 5 (A rating),Creative & Sporting
2,]performance s p a c e [,Folkestone,Kent,Tier 5 (A rating),Creative & Sporting
3,012 Global Ltd,London,Kent,Tier 2 (A rating),Tier 2 General
4,0-two Maintenance,London,Kent,Tier 2 (A rating),Tier 2 General


### Export CSV

In [59]:
all_data.to_csv('Register of Sponsors.csv', index=False)

### Sample search


Use regex matching to match a search keyword with Organization Name

In [ ]:
search_org1 = '012 global ltd'
search_org2 = '101'
search_org3 = 'london'

In [ ]:
def search_organization(keyword):    
    res = all_data['Organization Name'].loc[all_data['Organization Name'].str.contains(r'(?i)' + keyword, regex=True)]
    
    return res

In [ ]:
search_organization(search_org1)

In [ ]:
search_organization(search_org2)

In [ ]:
search_organization(search_org3)

### Get unique towns and counties

In [ ]:
def get_unique_place_names(column_data):
    non_null_data = column_data.drop(column_data.loc[column_data.isna()].index, axis=0)
    
    places = non_null_data.str.lower()
    unique_names = list(places.unique())
    
    for i, t in enumerate(unique_names):
        unique_names[i] = ' '.join(list(map(lambda x: x.capitalize(), t.split())))
        
    return sorted(unique_names)

In [ ]:
unique_towns = get_unique_place_names(all_data['Town/City'])
unique_counties = get_unique_place_names(all_data['County'])

In [ ]:
unique_towns[:10]

In [ ]:
unique_counties[:10]

In [ ]:
unique_tiers = get_unique_place_names(all_data['Tier & Rating'])
unique_sub_tiers = get_unique_place_names(all_data['Sub Tier'])

In [ ]:
unique_tiers

In [ ]:
unique_sub_tiers